In [41]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

In [42]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

In [43]:
def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

In [44]:
def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  

In [45]:
class_names = ["toi", "song", "truoc", "nhan_vien", "gia_dinh", "test_toi", "test_song", "test_truoc", 
               "test_nhan_vien", "test_gia_dinh"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("data", cname))

# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
print("vectors", all_vectors.shape)


Load toi dataset
Load song dataset
Load truoc dataset
Load nhan_vien dataset
Load gia_dinh dataset
Load test_toi dataset
Load test_song dataset
Load test_truoc dataset
Load test_nhan_vien dataset
Load test_gia_dinh dataset
vectors (18599, 36)


In [57]:
models = {}

for cname in class_names:
    # convert all vectors to the cluster index
    # dataset['one'] = [O^1, ... O^R]
    # O^r = (c1, c2, ... ct, ... cT)
    # O^r size T x 1
    hmm = hmmlearn.hmm.GMMHMM(
        n_components=6, n_mix = 2, random_state=42, n_iter=1000, verbose=True,
        params='mctw',
        init_params='m',
#         startprob_prior = np.array([1.0,0.0,0.0,0.0,0.0]),
#         transmat_prior = np.array([
#             [0.7,0.3,0.0,0.0,0.0],
#             [0.0,0.7,0.3,0.0,0.0],
#             [0.0,0.0,0.7,0.3,0.0],
#             [0.0,0.0,0.0,0.7,0.3],
#             [0.0,0.0,0.0,0.0,1.0],
#         ])
    )
    hmm.startprob_ = np.array([1.0,0.0,0.0,0.0,0.0, 0.0])
#     hmm.transmat_ = np.array([
#             [0.7,0.3,0.0,0.0,0.0],
#             [0.0,0.7,0.3,0.0,0.0],
#             [0.0,0.0,0.7,0.3,0.0],
#             [0.0,0.0,0.0,0.7,0.3],
#             [0.0,0.0,0.0,0.0,1.0],
#         ])

    if cname[:4] != 'test':
        X = np.concatenate(dataset[cname])
        lengths = list([len(x) for x in dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X)
        models[cname] = hmm
print("Training done")

training class toi
(2499, 36) [22, 36, 28, 39, 33, 23, 23, 21, 20, 23, 40, 48, 12, 36, 18, 25, 12, 22, 14, 23, 34, 22, 30, 10, 84, 47, 22, 29, 21, 25, 30, 17, 19, 31, 24, 27, 19, 38, 25, 21, 84, 24, 17, 17, 17, 16, 36, 17, 20, 18, 18, 32, 33, 29, 18, 47, 19, 20, 18, 24, 33, 16, 23, 31, 37, 17, 49, 31, 25, 30, 24, 82, 26, 17, 16, 57, 15, 24, 31, 16, 20, 20, 20, 49, 31, 34, 23, 22, 24, 39] 90


         1     -256218.3271             +nan
         2     -246103.1605      +10115.1667
         3     -243024.2634       +3078.8971
         4     -242466.5067        +557.7567
         5     -242157.3018        +309.2049
         6     -241952.8973        +204.4045
         7     -241786.3574        +166.5400
         8     -241630.7579        +155.5995
         9     -241515.9791        +114.7788
        10     -241422.4006         +93.5784
        11     -241346.0239         +76.3767
        12     -241262.0110         +84.0130
        13     -241178.3909         +83.6201
        14     -241128.7409         +49.6500
        15     -241096.1014         +32.6395
        16     -241073.2333         +22.8681
        17     -241038.3285         +34.9048
        18     -241021.5615         +16.7670
        19     -241011.1775         +10.3840
        20     -240990.6771         +20.5004
        21     -240938.0237         +52.6533
        22     -240893.3310         +44.6928
        23

training class song
(3110, 36) [25, 36, 32, 47, 31, 24, 22, 35, 54, 27, 35, 51, 34, 24, 25, 28, 17, 46, 32, 26, 31, 35, 30, 32, 28, 25, 31, 45, 30, 26, 43, 32, 45, 33, 31, 35, 41, 43, 33, 46, 32, 30, 47, 32, 47, 47, 42, 47, 41, 42, 33, 47, 36, 31, 24, 22, 36, 30, 33, 34, 32, 27, 44, 36, 45, 22, 30, 35, 31, 43, 35, 31, 54, 30, 45, 26, 25, 44, 22, 24, 17, 42, 26, 33, 42, 32, 31, 33, 43, 51] 90


         1     -321109.3421             +nan
         2     -302109.6508      +18999.6914
         3     -298456.7910       +3652.8598
         4     -297634.3213        +822.4697
         5     -297294.8086        +339.5127
         6     -297064.7765        +230.0321
         7     -296897.2264        +167.5502
         8     -296844.5697         +52.6566
         9     -296778.8754         +65.6944
        10     -296699.8447         +79.0307
        11     -296670.5278         +29.3169
        12     -296652.1836         +18.3442
        13     -296624.4125         +27.7711
        14     -296581.0901         +43.3224
        15     -296563.8538         +17.2363
        16     -296537.3255         +26.5284
        17     -296473.0288         +64.2967
        18     -296440.6440         +32.3848
        19     -296420.1172         +20.5268
        20     -296392.2571         +27.8601
        21     -296358.5056         +33.7515
        22     -296348.4674         +10.0382
        23

training class truoc
(2506, 36) [27, 20, 20, 41, 29, 31, 24, 26, 41, 21, 31, 26, 18, 23, 32, 23, 26, 21, 21, 37, 36, 26, 31, 15, 17, 19, 23, 26, 41, 25, 31, 31, 26, 31, 31, 26, 19, 35, 21, 22, 24, 20, 11, 31, 31, 22, 29, 33, 22, 29, 31, 22, 23, 24, 23, 31, 34, 21, 31, 26, 25, 20, 27, 19, 28, 27, 36, 23, 25, 35, 23, 31, 21, 41, 41, 36, 27, 61, 43, 43, 36, 35, 36, 20, 19, 19, 46, 31, 18, 26] 90


...

         1     -445167.9396             +nan
         2     -430219.9418      +14947.9978
         3     -426438.4006       +3781.5412
         4     -425483.8564        +954.5443
         5     -424948.5796        +535.2768
         6     -424698.2016        +250.3780
         7     -424521.1622        +177.0394
         8     -424350.1270        +171.0352
         9     -424182.6273        +167.4997
        10     -424051.9713        +130.6560
        11     -423958.6431         +93.3281
        12     -423877.6021         +81.0410
        13     -423819.9521         +57.6500
        14     -423779.8468         +40.1054
        15     -423745.8337         +34.0131
        16     -423715.0122         +30.8214
        17     -423688.8538         +26.1585
        18     -423665.6650         +23.1887
        19     -423643.9215         +21.7436
        20     -423624.0528         +19.8686
        21     -423609.9982         +14.0547
        22     -423598.3810         +11.6171
        23

training class gia_dinh
(4305, 36) [64, 51, 46, 55, 39, 43, 35, 66, 41, 41, 76, 47, 40, 46, 71, 45, 51, 40, 51, 41, 41, 53, 59, 47, 61, 37, 51, 51, 51, 47, 36, 44, 43, 46, 55, 46, 43, 39, 49, 44, 46, 51, 53, 37, 49, 33, 51, 51, 37, 46, 33, 41, 46, 66, 39, 37, 49, 45, 47, 31, 41, 41, 46, 45, 71, 43, 66, 55, 51, 56, 49, 51, 71, 33, 55, 35, 49, 41, 41, 51, 46, 51, 61, 45, 43, 61, 41, 41, 66, 39] 90


         1     -456030.0577             +nan
         2     -435333.1759      +20696.8819
         3     -431063.9761       +4269.1998
         4     -429503.7773       +1560.1988
         5     -428485.4497       +1018.3276
         6     -427879.1440        +606.3057
         7     -427541.7229        +337.4212
         8     -427246.3965        +295.3264
         9     -426930.6826        +315.7139
        10     -426755.7453        +174.9373
        11     -426676.7286         +79.0167
        12     -426608.7484         +67.9802
        13     -426541.5376         +67.2108
        14     -426513.2923         +28.2453
        15     -426492.2646         +21.0277
        16     -426475.7915         +16.4731
        17     -426457.4094         +18.3820
        18     -426437.9543         +19.4552
        19     -426426.0472         +11.9070
        20     -426414.8548         +11.1925
        21     -426400.5846         +14.2701
        22     -426378.1520         +22.4326
        23

Training done


        62     -426052.9684          +0.0164
        63     -426052.9562          +0.0122
        64     -426052.9474          +0.0088


In [58]:
print("Testing")
miss = {}
test_name = { "test_toi", "test_song", "test_truoc", "test_nhan_vien", "test_gia_dinh"}
for true_cname in test_name:
    kt = 0
    for O in dataset[true_cname]:
        score = {cname : model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test' }
        inverse = [(value, key) for key, value in score.items()]
        pre = max(inverse)[1]
        print(true_cname, score, pre)
        if pre != true_cname[5:]:
            kt +=1
    miss[true_cname] = kt
print(miss)

Testing
test_toi {'toi': -2575.2579869251044, 'song': -2886.6012050983154, 'truoc': -2905.0881778245434, 'nhan_vien': -2839.603606461546, 'gia_dinh': -2896.61400097453} toi
test_toi {'toi': -1963.4818990111246, 'song': -2217.9445967051997, 'truoc': -2151.2090723898787, 'nhan_vien': -2094.0786599404314, 'gia_dinh': -2078.0989088605434} toi
test_toi {'toi': -2013.3573678209852, 'song': -2297.655717868182, 'truoc': -2299.291925976029, 'nhan_vien': -2261.683543350359, 'gia_dinh': -2283.482908595709} toi
test_toi {'toi': -1895.5197172974747, 'song': -2191.5832205035485, 'truoc': -2176.172788027903, 'nhan_vien': -2185.05533377024, 'gia_dinh': -2171.0514014085165} toi
test_toi {'toi': -1677.7600932715518, 'song': -1850.1867517074718, 'truoc': -1760.5623116645309, 'nhan_vien': -1729.5154425177673, 'gia_dinh': -1810.8888235472652} toi
test_toi {'toi': -1711.6372863057734, 'song': -1864.3430171652087, 'truoc': -1799.6297107064672, 'nhan_vien': -1769.7494272992772, 'gia_dinh': -1729.9454778438812

In [59]:
models

{'toi': GMMHMM(algorithm='viterbi', covariance_type='diag',
        covars_prior=array([[[-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5],
         [-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5,
          -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -...
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]]]),
        means_weight=array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
        min_covar=0.001, n_components=6, n_iter=1000, n_mix=2, params='mctw',
        random_state=42, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
        verbose=True,
        weights_prior=array([[1., 1.],
 

In [60]:
print("Testing")
miss = {}
class_names = ["toi", "song", "truoc", "nhan_vien", "gia_dinh"]
for true_cname in class_names:
    kt = 0
    for O in dataset[true_cname]:
        score = {cname : model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test' }
        inverse = [(value, key) for key, value in score.items()]
        pre = max(inverse)[1]
        print(true_cname, score, pre)
        if pre != true_cname:
            kt +=1
    miss[true_cname] = kt
print(miss)

Testing
toi {'toi': -2291.7466300166006, 'song': -2403.4107995177574, 'truoc': -2541.4309478825426, 'nhan_vien': -2484.465534784773, 'gia_dinh': -2456.1811943801417} toi
toi {'toi': -3176.3855592396944, 'song': -3540.9398731753427, 'truoc': -3494.7395417734087, 'nhan_vien': -3343.434794416797, 'gia_dinh': -3466.4994404782383} toi
toi {'toi': -3150.086298327804, 'song': -3341.4358740880343, 'truoc': -3506.362782799021, 'nhan_vien': -3407.7893068862395, 'gia_dinh': -3387.0016497446436} toi
toi {'toi': -4035.609713826395, 'song': -4547.61940270488, 'truoc': -4581.427684032429, 'nhan_vien': -4472.463730802348, 'gia_dinh': -4531.93297615392} toi
toi {'toi': -2812.3223562800376, 'song': -3123.875142511453, 'truoc': -3104.3843232176914, 'nhan_vien': -3035.7120198487037, 'gia_dinh': -3085.579121820318} toi
toi {'toi': -2562.27758300875, 'song': -2872.81975361267, 'truoc': -3083.224051397845, 'nhan_vien': -3130.2588246404302, 'gia_dinh': -2837.5315335096498} toi
toi {'toi': -2115.9261990620853,

toi {'toi': -4752.944403014183, 'song': -5538.954458111365, 'truoc': -5577.995434928483, 'nhan_vien': -5457.768269423915, 'gia_dinh': -5303.178380704434} toi
toi {'toi': -2913.388253652058, 'song': -3101.291001606235, 'truoc': -3111.6823146078505, 'nhan_vien': -3158.838652063595, 'gia_dinh': -3035.3700575572266} toi
toi {'toi': -2324.8534727585784, 'song': -2616.1729934603504, 'truoc': -2606.9031340204047, 'nhan_vien': -2575.7380636009552, 'gia_dinh': -2560.9467259503094} toi
toi {'toi': -2930.4778086505953, 'song': -3349.558146679045, 'truoc': -3395.1643232366564, 'nhan_vien': -3294.3246133189737, 'gia_dinh': -3251.1715354500047} toi
toi {'toi': -2256.9554831961314, 'song': -2394.8196328859094, 'truoc': -2438.784147724401, 'nhan_vien': -2447.668116772853, 'gia_dinh': -2382.619119543042} toi
toi {'toi': -8116.698106155907, 'song': -9265.061371456795, 'truoc': -9278.094519522181, 'nhan_vien': -9165.62205383357, 'gia_dinh': -8975.0065119643} toi
toi {'toi': -2516.9387807457756, 'song': -

song {'toi': -3962.407687804162, 'song': -3701.3145250364014, 'truoc': -3979.750097539511, 'nhan_vien': -4046.3887604326464, 'gia_dinh': -3864.412851329241} song
song {'toi': -4772.833217436826, 'song': -4085.755416577093, 'truoc': -4766.7562713354655, 'nhan_vien': -4993.098304087572, 'gia_dinh': -4730.6657427857135} song
song {'toi': -3737.7584141549873, 'song': -3261.643324426201, 'truoc': -3723.3931937643315, 'nhan_vien': -3830.719719365274, 'gia_dinh': -3688.1863317900393} song
song {'toi': -5001.529572342241, 'song': -4422.325385846511, 'truoc': -5333.83329452965, 'nhan_vien': -5329.95057484479, 'gia_dinh': -5014.5872637657385} song
song {'toi': -3590.2347541229647, 'song': -3311.4763738290935, 'truoc': -3588.5373819350903, 'nhan_vien': -3710.6258318339524, 'gia_dinh': -3556.1486635869874} song
song {'toi': -3090.9831521763763, 'song': -2964.857016741936, 'truoc': -3107.70675846842, 'nhan_vien': -3256.083354983954, 'gia_dinh': -3134.21716254373} song
song {'toi': -2570.58723172804

truoc {'toi': -3664.364755188321, 'song': -3649.444415683891, 'truoc': -3362.3344037781767, 'nhan_vien': -3641.886148070375, 'gia_dinh': -3672.3955855616914} truoc
truoc {'toi': -3116.066934522729, 'song': -3033.3244552340334, 'truoc': -2765.4641607891763, 'nhan_vien': -3170.2806954193065, 'gia_dinh': -3062.6516729215114} truoc
truoc {'toi': -4339.9269096560665, 'song': -4471.480537138364, 'truoc': -3639.2846908679203, 'nhan_vien': -4430.839783853841, 'gia_dinh': -4113.929784418102} truoc
truoc {'toi': -1706.8861373863306, 'song': -1736.8854738107739, 'truoc': -1615.986855541245, 'nhan_vien': -1768.347789995628, 'gia_dinh': -1741.332410784274} truoc
truoc {'toi': -1561.5400116355827, 'song': -1676.801298851658, 'truoc': -1580.451074509367, 'nhan_vien': -1603.2820253105137, 'gia_dinh': -1691.8667302237413} toi
truoc {'toi': -1821.3001050659502, 'song': -1883.7356742882218, 'truoc': -1726.8037110506957, 'nhan_vien': -1859.2483085952074, 'gia_dinh': -1868.572517535441} truoc
truoc {'toi':

nhan_vien {'toi': -4968.371296825966, 'song': -5135.710125668406, 'truoc': -4895.874702968547, 'nhan_vien': -4522.759546693713, 'gia_dinh': -4912.876459580086} nhan_vien
nhan_vien {'toi': -5528.960237183881, 'song': -5710.439622595422, 'truoc': -5598.315710107319, 'nhan_vien': -5195.496798798766, 'gia_dinh': -5529.3855138214} nhan_vien
nhan_vien {'toi': -6311.353489792331, 'song': -6493.576798669935, 'truoc': -6104.54355431481, 'nhan_vien': -5732.711935253173, 'gia_dinh': -6220.197813653883} nhan_vien
nhan_vien {'toi': -2827.549460099553, 'song': -2948.098361500456, 'truoc': -2825.523378565767, 'nhan_vien': -2584.7916679256577, 'gia_dinh': -2809.167393879849} nhan_vien
nhan_vien {'toi': -3286.3447762876253, 'song': -3525.6802985167405, 'truoc': -3331.783853335793, 'nhan_vien': -3012.3003561602804, 'gia_dinh': -3320.5698917602917} nhan_vien
nhan_vien {'toi': -5135.426104033266, 'song': -5268.632558380446, 'truoc': -5055.29198404603, 'nhan_vien': -4688.35887841153, 'gia_dinh': -5119.5174

nhan_vien {'toi': -3883.5506286557143, 'song': -4430.081474262946, 'truoc': -4171.802776550267, 'nhan_vien': -3863.56003362673, 'gia_dinh': -4099.411431129683} nhan_vien
nhan_vien {'toi': -3837.4640122256965, 'song': -3976.473219154471, 'truoc': -3951.448741116141, 'nhan_vien': -3519.7364783085327, 'gia_dinh': -3769.2843491510016} nhan_vien
nhan_vien {'toi': -3414.2953006737184, 'song': -3637.2679315134765, 'truoc': -3615.942045090149, 'nhan_vien': -3277.2346623618414, 'gia_dinh': -3466.8416537806306} nhan_vien
nhan_vien {'toi': -4645.65374745362, 'song': -4800.039116117276, 'truoc': -4749.794675098529, 'nhan_vien': -4438.7576526433595, 'gia_dinh': -4550.0364667763215} nhan_vien
nhan_vien {'toi': -4846.608519432307, 'song': -5123.542390204827, 'truoc': -5002.683333099954, 'nhan_vien': -4749.457337278534, 'gia_dinh': -4963.5197703805115} nhan_vien
nhan_vien {'toi': -3601.970924924231, 'song': -3834.8027672097764, 'truoc': -3588.7193070063436, 'nhan_vien': -3359.0785120471164, 'gia_dinh'

gia_dinh {'toi': -5015.320402727232, 'song': -5128.018538642725, 'truoc': -5122.846522294834, 'nhan_vien': -4992.772290353982, 'gia_dinh': -4248.171108295482} gia_dinh
gia_dinh {'toi': -5043.217884380317, 'song': -5407.256776040594, 'truoc': -5339.846513021502, 'nhan_vien': -5276.150061972608, 'gia_dinh': -4340.460682931601} gia_dinh
gia_dinh {'toi': -5658.123574990759, 'song': -5800.450268290308, 'truoc': -5699.815832790783, 'nhan_vien': -5531.879129762126, 'gia_dinh': -5158.349108229708} gia_dinh
gia_dinh {'toi': -6165.541030311036, 'song': -6422.844575230469, 'truoc': -6534.279072265572, 'nhan_vien': -6228.689976408775, 'gia_dinh': -5532.179945528209} gia_dinh
gia_dinh {'toi': -5068.654269547844, 'song': -5352.689029021559, 'truoc': -5159.188032240842, 'nhan_vien': -5148.464488562815, 'gia_dinh': -4730.181735681103} gia_dinh
gia_dinh {'toi': -6321.42902595574, 'song': -6602.131251521227, 'truoc': -6573.100873692891, 'nhan_vien': -6288.047613336856, 'gia_dinh': -5981.854568033459} gi

gia_dinh {'toi': -3374.690280005665, 'song': -3553.9225158199556, 'truoc': -3620.1455008387543, 'nhan_vien': -3337.5261600836793, 'gia_dinh': -3101.577639688273} gia_dinh
gia_dinh {'toi': -5695.567663607749, 'song': -5769.498780926242, 'truoc': -5799.669874052994, 'nhan_vien': -5686.86100114345, 'gia_dinh': -5405.918101807438} gia_dinh
gia_dinh {'toi': -3654.4176663089434, 'song': -3735.610079329734, 'truoc': -3745.0976186363787, 'nhan_vien': -3611.2863064204766, 'gia_dinh': -3335.4913343445605} gia_dinh
gia_dinh {'toi': -4949.740670549249, 'song': -5074.179024395451, 'truoc': -5169.005920009587, 'nhan_vien': -4809.49027145333, 'gia_dinh': -4519.256961756482} gia_dinh
gia_dinh {'toi': -4403.708204660438, 'song': -4527.7160760666675, 'truoc': -4537.68074568282, 'nhan_vien': -4365.929309322025, 'gia_dinh': -3971.3671253070916} gia_dinh
gia_dinh {'toi': -5202.691265614639, 'song': -5767.399019648715, 'truoc': -5485.708262004285, 'nhan_vien': -5312.877880239846, 'gia_dinh': -4449.353939723

In [61]:
import pickle 
with open("output.pkl", "wb") as file:
    pickle.dump(models, file)

In [56]:
!ls

 assignment1
 assignment2-data.csv
 data
'Điểm danh Tran Quoc Long - INT3411 1 - Xu ly tieng noi.xlsx'
'dowload file.ipynb'
'get data.ipynb'
'gia đình.xlsx'
'nhân viên.xlsx'
 output.pkl
 sống.xlsx
'speech recognition.ipynb'
 tôi.xlsx
 trước.xlsx
 Untitled.ipynb
